<a href="https://colab.research.google.com/github/mohammedhemed77/NN-zero-to-hero-personal-practice/blob/main/makemore_part1_bigrams_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#E01: train a trigram language model, i.e.
# take two characters as an input to predict the 3rd one.
# Feel free to use either counting or a neural net.
# Evaluate the loss; Did it improve over a bigram model?

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# load the dataset into massive string
words = open('names.txt','r').read().splitlines()

for w in words[:1]:
  for ch1,ch2 in zip(w , w[1:]):
    print (ch1,ch2)

# display first 7 words
words [:7]

print ("dataset words = " , len(words))
print ("minimum word length = ",min (len (w) for w in words))
print ("maximum word length = ",max (len (w) for w in words))


e m
m m
m a
dataset words =  32033
minimum word length =  2
maximum word length =  15


In [ ]:
# using counters
N = torch.ones((27,27,27),dtype = torch.float32)
N[0,0,0] = 0
# concat all words in single string
# concatenate all words into single string
massive_string = list (''.join(words))
print (len(massive_string))
print (len(words))
chars = sorted(list(set(''.join(words))))

# mapping chars into integers and integrs into chars via 2 dic :
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

196113
32033


In [ ]:
for w in words :
  chs = ['.'] + list(w) + ['.']
  for ch1 , ch2 , ch3 in zip (chs , chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1,ix2,ix3] += 1

P = N / N.sum(dim =2, keepdim = True)

In [ ]:
# loss :
def calculate_loss(input_set):
  log_likelihood = 0.0
  n = 0
  for w in input_set :
    chs = ['.'] + list(w) + ['.']
    for ch1 , ch2 , ch3 in zip(w,w[1:],w[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]

      prob = P[ix1,ix2,ix3]
      log_prob = torch.log(prob)
      log_likelihood += log_prob
      n+=1
  nll = -log_likelihood
  print (f'log_likelihood = {log_likelihood}')
  print (f'negative log likelihood = {nll} ')
  print (f'Average negative log likelihood = {nll/n}')


print (f'training loss : ')
calculate_loss(words)


training loss : 
log_likelihood = -307831.9375
negative log likelihood = 307831.9375 
Average negative log likelihood = 2.3312301635742188


In [ ]:
# using NN
# Prepare the training set
xs , ys = [], []

for w in words:
    # add start and end tokens
    chs = ["."] + list(w) + ["."]
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        #print (itos[ix1],itos[ix2],itos[ix3],"\n")
        xs.append([ix1, ix2])
        ys.append(ix3)

xs = torch.tensor(xs, dtype=torch.int64)
ys = torch.tensor(ys, dtype=torch.int64)


print (xs.shape)
print (ys.shape)



torch.Size([196113, 2])
torch.Size([196113])


In [ ]:
W = torch.randn((27*2,27), requires_grad = True)
for k in range(200):
    # forward pass
    xenc = F.one_hot(xs, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)
    # probs is softmax
    logits = xenc @ W
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)

    # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(xs)), ys].log().mean()
    # add regularization
    loss += 0.2 * W.pow(2).mean()

    if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

    # backward pass
    W.grad = None
    loss.backward()

    # update weights
    with torch.no_grad():
        W -= 50 * W.grad

0: 4.4154
10: 0.2874
20: 0.2465
30: 0.2208
40: 0.2022
50: 0.1884
60: 0.1780
70: 0.1703
80: 0.1645
90: 0.1602
100: 0.1569
110: 0.1545
120: 0.1526
130: 0.1513
140: 0.1502
150: 0.1494
160: 0.1488
170: 0.1484
180: 0.1481
190: 0.1478


In [ ]:
names = []
for i in range(10):
    out = []
    ix1, ix2 = 0, 0
    while True:
        # previosly we used P[ix]
        # p = P[ix]

        # now we use the softmax of the logits
        xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes = 27).float()
        xenc = xenc.view(-1, 27*2)

        logits = xenc @ W
        counts = torch.exp(logits)
        p = counts / counts.sum(dim = 1, keepdim = True)

        ix1 = ix2
        ix2 = torch.multinomial(p, num_samples = 1 , replacement = True).item()
        out.append(itos[ix2])
        if ix2 == 0:
            break

    names.append("".join(out))

for name in names:
    print(name)

calculate_loss(names)

In [ ]:
a = torch.range(1,16)
print (a)
a = a.view(-1,4)
print (a.view(4,-1))
print(a)

In [ ]:
print (W.shape)
print(xenc.shape)


torch.Size([54, 27])
torch.Size([196113, 54])


In [32]:
import torch
import torch.nn.functional as F
indices = torch.tensor ([3,4,5,6,8])
# num_classes = 5
one_hot_encoded = F.one_hot(indices,num_classes=10)

print (f'indices ={indices} ')
print (f'one_hot_encoded = {one_hot_encoded}')


indices =tensor([3, 4, 5, 6, 8]) 
one_hot_encoded = tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])


***E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set***

In [39]:
from sklearn.model_selection import train_test_split

words_train , words_test = train_test_split(words,test_size=0.2,random_state=1234)

words_dev , words_test   = train_test_split(words_test,test_size=0.5,random_state=1234)

x_train , y_train , x_dev , y_dev , x_test , y_test = [],[],[],[],[],[]

# list of lists

for wgroup in [words_train,words_dev,words_test]:
  xs , ys = [],[]
  for w in wgroup:
    chs = ['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      xs.append([ix1,ix2])
      ys.append(ix3)

  xs = torch.tensor(xs,dtype=torch.int64)
  ys = torch.tensor(ys,dtype=torch.int64)

  if wgroup == words_train : x_train,y_train = xs , ys
  elif wgroup == words_dev : x_dev , y_dev   = xs , ys
  else: x_test,y_test = xs,ys



In [41]:
for k in range(200):
    # forward pass
    xenc = F.one_hot(x_train, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)

    # probs is softmax
    logits = xenc @ W
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)

    # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(x_train)), y_train].log().mean()
    # add regularization
    # loss += 0.2 * W.pow(2).mean()

    if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

    # backward pass
    W.grad = None
    loss.backward()

    # update weights
    with torch.no_grad():
        W -= 50 * W.grad

0: 4.9474
10: 2.3903
20: 2.3183
30: 2.2936
40: 2.2810
50: 2.2731
60: 2.2676
70: 2.2635
80: 2.2604
90: 2.2579
100: 2.2558
110: 2.2541
120: 2.2526
130: 2.2514
140: 2.2503
150: 2.2493
160: 2.2485
170: 2.2477
180: 2.2471
190: 2.2464


In [42]:
def MLP_loss(x, y, W):
    xenc = F.one_hot(x, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)

    # probs is softmax
    logits = xenc @ W
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)

    # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(x)), y].log().mean()
    return loss.item()

In [43]:
print(f"Train Loss: {MLP_loss(x_train, y_train, W):.4f}")
print(f"Dev Loss: {MLP_loss(x_dev, y_dev, W):.4f}")
print(f"Test Loss: {MLP_loss(x_test, y_test, W):.4f}")

Train Loss: 2.2459
Dev Loss: 2.2510
Test Loss: 2.2489


***E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model*** *italicised text*

In [45]:
W = torch.randn((27*2,27), requires_grad = True)
for k in range(200):
    # forward pass
    xenc = F.one_hot(x_train, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)

    # probs is softmax
    logits = xenc @ W
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)

    # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(x_train)), y_train].log().mean()
    # add regularization
    # loss += 0.05 * W.pow(2).mean()

    if k % 10 == 0:
        print(f"{k}: Train Loss: {loss.item():.4f} | Dev Loss {MLP_loss(x_dev, y_dev, W):.4f}")

    # backward pass
    W.grad = None
    loss.backward()

    # update weights
    with torch.no_grad():
        W -= 50 * W.grad

0: Train Loss: 4.2715 | Dev Loss 4.2730
10: Train Loss: 2.4919 | Dev Loss 2.5001
20: Train Loss: 2.3737 | Dev Loss 2.3790
30: Train Loss: 2.3283 | Dev Loss 2.3327
40: Train Loss: 2.3049 | Dev Loss 2.3089
50: Train Loss: 2.2907 | Dev Loss 2.2945
60: Train Loss: 2.2812 | Dev Loss 2.2850
70: Train Loss: 2.2744 | Dev Loss 2.2782
80: Train Loss: 2.2693 | Dev Loss 2.2732
90: Train Loss: 2.2653 | Dev Loss 2.2692
100: Train Loss: 2.2621 | Dev Loss 2.2661
110: Train Loss: 2.2594 | Dev Loss 2.2636
120: Train Loss: 2.2573 | Dev Loss 2.2615
130: Train Loss: 2.2554 | Dev Loss 2.2598
140: Train Loss: 2.2539 | Dev Loss 2.2583
150: Train Loss: 2.2525 | Dev Loss 2.2570
160: Train Loss: 2.2513 | Dev Loss 2.2559
170: Train Loss: 2.2503 | Dev Loss 2.2550
180: Train Loss: 2.2493 | Dev Loss 2.2541
190: Train Loss: 2.2485 | Dev Loss 2.2534


***E04: Rewrite the MLP model without creating one hot vectors
we saw that our 1-hot vectors merely select a row of W,
 so producing these vectors explicitly feels wasteful.
 Can you delete our use of F.one_hot in favor of simply indexing into rows of W?***

In [47]:
W = torch.randn((27*2,27), requires_grad = True)
for k in range(200):
    # forward pass
    # ====================
    # Previously: using onehot and multiplying by W
    # xenc = F.one_hot(xs, num_classes = 27).float().to(device)
    # xenc = xenc.view(-1, 27*2)
    # logits = xenc @ W
    # ====================

    # ====================
    # ✅ now: acess by xs indices directly
    logits = W[xs[:,0]] + W[xs[:,1] + 27]
    # ====================

    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)
     # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(xs)), ys].log().mean()
    # add regularization
    loss += 0.2 * W.pow(2).mean()

    if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

    # backward pass
    W.grad = None
    loss.backward()

    # update weights
    with torch.no_grad():
        W -= 50 * W.grad

0: 4.2341
10: 2.5841
20: 2.4541
30: 2.4070
40: 2.3841
50: 2.3715
60: 2.3641
70: 2.3594
80: 2.3564
90: 2.3544
100: 2.3530
110: 2.3521
120: 2.3515
130: 2.3510
140: 2.3507
150: 2.3505
160: 2.3503
170: 2.3502
180: 2.3501
190: 2.3500


***E05: look up and use F.cross_entropy instead
nn.functonal.cross_entropy() takes the logits and the target class as input and returns the cross entropy loss directly
***

In [50]:
W = torch.randn((27*2,27), requires_grad = True)
for k in range(200):
    # forward pass
    xenc = F.one_hot(xs, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)
    logits = xenc @ W

    loss = torch.nn.functional.cross_entropy(logits, ys)
    # add regularization
    loss += 0.2 * W.pow(2).mean()

    if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

    # backward pass
    W.grad = None
    loss.backward()

    # update weights
    with torch.no_grad():
        W -= 50 * W.grad

0: 4.4883
10: 2.5857
20: 2.4573
30: 2.4099
40: 2.3864
50: 2.3731
60: 2.3651
70: 2.3600
80: 2.3568
90: 2.3546
100: 2.3532
110: 2.3522
120: 2.3515
130: 2.3510
140: 2.3507
150: 2.3505
160: 2.3503
170: 2.3502
180: 2.3501
190: 2.3500
